In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'operator_login', 'Дата и время звонка': 'call_date', 
                'Результат звонка': 'call_result', 'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 
                'Комменатрий': 'comment', 'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 
                'ФИО врача': 'doctor_fio', 'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'operator_login',
                'Зал №': 'hall_number', 'Логин naumen': 'operator_login', 'МО закрепления': 'mo', 
                'Отсутствует на рабочем месте': 'absence', 'login': 'operator_login', 'procedure_rec_status': 'group_procedure',
                'patient_group': 'group_name'}

In [3]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_21


In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = \
r'\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses'

patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_data = os.path.join('C:\\', r'\\', *patients_current_statuses_path,
                                             f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx')
print(patients_current_statuses_data)

C:\\Users\murad_satabaev\Documents\GitHub\final_framework\merged_statuses\total_patients_current_statuses_merged_with_logins_2022_09_21.xlsx


In [5]:
# путь файла с сегодняшними 
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)
print(emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx


In [6]:
today_emias_data_path = [os.path.join(emias_data_path, filename) for filename in os.listdir(emias_data_path) if '4059' 
                         in filename and '~' not in filename and today.strftime('%Y-%m-%d') in filename][0]
print(today_emias_data_path)

\\t999\Сетевой диск\pdn\data\emias_data\xlsx\DAT_4059_выгрузка_списков_на_обзвон_помощниками_2022-09-21.xlsx


In [7]:
# чтение всех файлов по отдельности, чтобы не тратить время на повторное чтение файла, который уже прочитан

In [8]:
today_emias_data_reading = pd.read_excel(today_emias_data_path)
today_emias_data_reading.shape[0]

97170

In [9]:
patients_current_statuses_reading = pd.read_excel(patients_current_statuses_data)
patients_current_statuses_reading.shape[0]

109384

In [10]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


print(dt.datetime.now())
print(assistants_presence_reading.shape[0])

assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()

2022-09-21 09:19:06.328894
217


In [11]:
if assistants_presence[assistants_presence.absence.isna()].operator_login.nunique() + \
assistants_presence[assistants_presence.absence == 1].operator_login.nunique() == assistants_presence.operator_login.nunique():
    print('\nВ сумме количество отсуствующих и присутствующих равны общему количеству помощников\n')
else:
    print(5 * '\nПРОВЕРЬТЕ ТАБЛИЦУ ОТСУТСТВУЮЩИХ И ПРИСУТСТВУЮЩИХ\n!!!\n')


В сумме количество отсуствующих и присутствующих равны общему количеству помощников



In [12]:
present_assistants = assistants_presence[assistants_presence.absence.isna()]
print(present_assistants.shape[0])
absent_assistants = assistants_presence[~assistants_presence.absence.isna()]
print(absent_assistants.shape[0])

164
53


In [13]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()

In [14]:
today_emias_data = today_emias_data_reading.rename(columns=column_names)

In [15]:
patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
patients_current_statuses['next_call_date'] = patients_current_statuses['next_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
patients_current_statuses['last_call_date'] = patients_current_statuses['last_call_date'] \
.apply(lambda x: pd.to_datetime(x, dayfirst=True).date())

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\1251941699.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')


In [ ]:
# Переведем все типы данных емиасайди, телефоны и прочие числовые данные в строки

In [16]:
today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\3971369908.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['emiasid'] = today_emias_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\3971369908.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  today_emias_data['phone'] = today_emias_data['phone'].astype(str).str.replace('\.0', '')


In [17]:
# путь папки с раскреплением
assistant_to_doctor_distribution_path = \
r'\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path'
assistant_to_doctor_distribution_path = \
assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = \
os.path.join(r'\\', *assistant_to_doctor_distribution_path)
print(assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path


In [18]:
september_assistant_to_doctor_distribution_path = [os.path.join(assistant_to_doctor_distribution_path, filename) for filename 
                                         in os.listdir(assistant_to_doctor_distribution_path) if '~' not in filename
                                        and 'september' in filename]
september_assistant_to_doctor_distribution_path.sort(key=lambda x: os.path.getctime(x))
september_assistant_to_doctor_distribution_path = september_assistant_to_doctor_distribution_path[-1]
print(september_assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path\september_patient_to_assistant_distribution.xlsx


In [19]:
september_assistant_to_doctor_distribution = pd.read_excel(september_assistant_to_doctor_distribution_path, 
                                    usecols=['login', 'emiasid', 'mo', 'doctor_fio'])[['emiasid', 'login', 'mo', 'doctor_fio']]

In [20]:
doctors_list = set(september_assistant_to_doctor_distribution.doctor_fio.unique())
doctors_list.add(np.NaN)

In [21]:
def find_phone_number(text):
    """Функция ищет номера телефонов в строке, фильтрует дефектные и возвращает один с приоритетом на мобильный"""
    import re
    defect_pattern = re.compile(r"""7[^94]\d{9}|0{7,}|1{7,}|2{7,}|3{7,}|4{7,}|5{7,}|6{7,}|7{7,}|8{7,}|9{7,}""")
    text = str(text)
    text = ['7' + re.sub(r'\D', '', x)[-10:] for x in re.split(r'[,;]', text)]
    text = list(filter(lambda x: len(x) == 11, text))
    text = list(filter(lambda x: False if re.search(defect_pattern, x) else True, text))
    text = sorted(text, reverse=True)
    text = ','.join(text)
    text = re.search(r'(9\d{9}|4\d{9})', text)
    if text:
        text = text.group()
    return text

today_emias_data['phone_1'] = today_emias_data['phone'].apply(find_phone_number)
today_emias_data[today_emias_data['phone_1'].isna()].shape[0]

321

In [22]:
today_emias_data.shape[0]

97170

109384

In [27]:
patients_current_statuses.columns

Index(['Unnamed: 0', 'emiasid', 'status', 'old_comment', 'next_call_date',
       'operator_login', 'mo', 'doctor_fio', 'calls_amount', 'last_call_date',
       'last_call_result', 'last_talk_result', 'month', 'problem', 'group',
       'group_procedure', 'group_name', 'absence', 'is_in_today_task'],
      dtype='object')

In [24]:
exception = pd.read_excel('exception.xlsx')

In [25]:
exception = exception.rename(columns=column_names)
exception['emiasid'] = exception['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\4054622638.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  exception['emiasid'] = exception['emiasid'].astype(str).str.replace('\.0', '')


In [26]:
exception.shape[0]

372

In [ ]:
patients_current_statuses.problem.unique()

In [157]:
actual_patients = \
patients_current_statuses[
    (~(patients_current_statuses.problem.isin({
    'Нет в сегодняшней выгрузке', 'Нет прикрепления к МО', 'Отсутствует номер', 'Смена МО', 'Смена МО на другую нашу'
    })))
    &
    (~(patients_current_statuses.emiasid.isin(exception.emiasid)))
    &
    (patients_current_statuses.is_in_today_task == 'нет')
]

actual_patients.shape[0]

92973

In [29]:
patients_current_statuses.shape[0]

109384

In [30]:
actual_patients.shape[0]

92973

In [31]:
actual_patients.status.value_counts()

Не дозвонились                    35641
Перенос звонка на другой день     27097
Будет в Москве в другом месяце     9942
Разговор состоялся                 8127
Ни разу не звонили                 3668
Запишется самостоятельно           2674
Перезвонить сегодня                1504
Отказ                              1004
Пациент отрицает диагноз            958
Отсутствует номер                   601
Иная МО                             531
Госпитализирован                    435
Живет не в Москве                   289
Патронаж                            228
Некорректный номер                  121
Не нравится врач                     66
Другой врач в МО                     36
Онкология                            23
Снят с ДН                            17
Не нравится МО                       10
Умер                                  1
Name: status, dtype: int64

In [32]:
actual_patients.operator_login.count()

90944

In [33]:
actual_patients.loc[actual_patients.problem == 'Новый врач', 'operator_login'] = np.nan

In [34]:
actual_patients.operator_login.count()

82778

In [35]:
first_stage_patients = \
actual_patients[
    (actual_patients.status == 'Ни разу не звонили')
    &
    (actual_patients.month == 'september')
]

first_stage_patients['priority'] = 2
first_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\3473140784.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_stage_patients['priority'] = 2


11

In [37]:
call_back_on_date_patients = \
actual_patients[
    (actual_patients.next_call_date.isin({
        dt.date(2022, 9, 16),
        dt.date(2022, 9, 19),
        dt.date(2022, 9, 20),
        dt.date(2022, 9, 21)
    }))
]

call_back_on_date_patients['priority'] = 1
call_back_on_date_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\3383705759.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_back_on_date_patients['priority'] = 1


4503

In [ ]:
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
].shape[0]

In [ ]:
actual_patients.last_call_result.unique()

In [42]:
august_patients = \
actual_patients[
    (actual_patients.month == 'august')
    &
    (actual_patients.last_call_result != 'Перенос звонка на другой день')
    &
    (actual_patients.last_call_date < dt.date(2022, 9, 2))
]

august_patients = august_patients.head(800)

august_patients['priority'] = 3
august_patients.shape[0]

800

In [43]:
august_patients.last_call_date.unique()[-1]

datetime.date(2022, 8, 26)

In [44]:
tomorrow_record_patients_set = set()
for column in today_emias_data.columns[22:98:2]:
    tomorrow_record_patients_set = \
    tomorrow_record_patients_set | set(today_emias_data[today_emias_data[f'{column}'] == tomorrow_y_m_d]\
                                   .emiasid.astype(str).unique())

print(len(tomorrow_record_patients_set))

4407


In [45]:
tomorrow_record_patients = patients_current_statuses[(patients_current_statuses.emiasid.isin(tomorrow_record_patients_set))
& (~(patients_current_statuses.status.isin({'Ни разу не звонили', 'Онкология', 'Смена МО', 'Отсутствует номер', 
                                           'Пациент отрицает диагноз', 'Отказ', 'Госпитализирован',
                                           'Будет в Москве в другом месяце', 'Некорректный номер', 'Патронаж',
                                           'Снят с ДН'}))) & 
(patients_current_statuses.last_call_result != 'Перенос звонка на другой день')]
print(tomorrow_record_patients.shape[0])

1890


In [46]:
tomorrow_record_patients['priority'] = 4

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\1903791372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomorrow_record_patients['priority'] = 4


In [47]:
tomorrow_record_patients.status.value_counts()

Не дозвонились                   1181
Разговор состоялся                576
Перезвонить сегодня                65
Запишется самостоятельно           47
Перенос звонка на другой день      14
Иная МО                             2
Не нравится МО                      2
Живет не в Москве                   1
Не нравится врач                    1
Другой врач в МО                    1
Name: status, dtype: int64

In [48]:
tomorrow_record_patients['status'] = 'Напомнить'

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\672541353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tomorrow_record_patients['status'] = 'Напомнить'


In [107]:
second_stage_patients = \
actual_patients[
    (actual_patients.status.isin({'Не дозвонились', 'Запишется самостоятельно', 'Не нравится МО', 'Другой врач в МО',
                                 'Не нравится врач'}))
    &
    (~(actual_patients.emiasid.isin(call_back_on_date_patients.emiasid)))
    &
    (~(actual_patients.last_call_date.isin(
        {
        dt.date(2022, 9, 19)
        }
    )))
]

second_stage_patients['priority'] = 5

second_stage_patients.shape[0]

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\3179676984.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_stage_patients['priority'] = 5


32207

In [108]:
final_actual_patients_for_task_ = pd.concat([
    call_back_on_date_patients,
    first_stage_patients,
    august_patients,
    second_stage_patients,
    tomorrow_record_patients
]) \
.sort_values(['priority']).reset_index(drop=True)

final_actual_patients_for_task_.shape[0]

39411

In [109]:
# final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['phone_number'], keep='first')
# final_actual_patients_for_task.shape[0]

In [110]:
final_actual_patients_for_task = final_actual_patients_for_task_.drop_duplicates(subset=['emiasid'], keep='first')
final_actual_patients_for_task.shape[0]

38547

In [111]:
final_actual_patients_for_task.head(3)

,Unnamed: 0,emiasid,status,old_comment,next_call_date,operator_login,mo,doctor_fio,calls_amount,last_call_date,last_call_result,last_talk_result,month,problem,group,group_procedure,group_name,absence,is_in_today_task,priority
0,24,10253534,Перенос звонка на другой день,NaN,2022-09-21,pomoshnik_pdn172,ДЦ 5,Желудкова Лидия Николаевна,6,2022-09-19,Перенос звонка на другой день,NaN,august,NaN,13.0,выполненная запись,Записать на прием,0,нет,1
1,79261,2245166168,Перенос звонка на другой день,перезвонить,2022-09-21,pomoshnik_pdn240,ГП 68,Юрга Надежда Христофоровна,2,2022-09-12,Перенос звонка на другой день,NaN,september,NaN,10.0,надо записать,Записать на прием - Записать на исследования -...,0,нет,1
2,79278,2245179714,Перенос звонка на другой день,"Нет слотов, 13380",2022-09-20,pomoshnik_pdn93,ГП 134,Бердышева Евгения Вячеславовна,1,2022-09-09,Перенос звонка на другой день,NaN,september,NaN,4.0,в будущем,Записать на прием,1,нет,1


In [112]:
final_actual_patients_for_task = \
final_actual_patients_for_task[['emiasid', 'status', 'old_comment', 'operator_login', 'mo', 'doctor_fio', 'priority', 'calls_amount',
                                'last_call_date', 'month']]

In [113]:
final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\2998909243.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  final_actual_patients_for_task['emiasid'] = final_actual_patients_for_task['emiasid'].astype(str).str.replace('\.0', '')


In [114]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task.copy()

In [115]:
initial_assistant_to_patient_distribution_in_today_task.old_comment.nunique()

4159

In [116]:
initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(
    ['mo', 'emiasid']).head(20)

emiasid
mo     operator_login           
ГП 107 pomoshnik_pdn131        1
       pomoshnik_pdn130       30
       pomoshnik_pdn148      163
       pomoshnik_pdn43       172
       pomoshnik_pdn109      179
       pomoshnik_pdn129      221
       pomoshnik_pdn185      228
ГП 109 pomoshnik_pdn155        5
       pomoshnik_pdn134      132
       pomoshnik_pdn168      148
ГП 11  pomoshnik_pdn32        54
       pomoshnik_pdn146      155
       pomoshnik_pdn147      253
       pomoshnik_pdn184      285
ГП 134 pomoshnik_pdn89       236
       pomoshnik_pdn7        273
       pomoshnik_pdn93       298
       pomoshnik_pdn75       319
       Помощник 3            364
ГП 166 pomoshnik_pdn209       33

In [117]:
set(present_assistants.operator_login.unique()) - \
(set(initial_assistant_to_patient_distribution_in_today_task.operator_login.unique()) - {np.nan})

{'pomoshnik_pdn151', 'pomoshnik_pdn99', 'Помощник 10'}

In [118]:
# initial_assistant_to_patient_distribution_in_today_task.to_excel('initial_assistant_to_patient_distribution_in_today_task.xlsx')

In [119]:
task_amount = 90

In [120]:
%%time

initial_task_for_today = pd.DataFrame()
for operator_login in set(initial_assistant_to_patient_distribution_in_today_task.operator_login):
    initial_assistant_to_patient_distribution_in_today_task_for_operator_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task \
                                                            .operator_login
         == operator_login].sort_values(by=['priority', 'calls_amount', 'last_call_date'])
    call_back_number_for_operator_login = initial_assistant_to_patient_distribution_in_today_task_for_operator_login[
        initial_assistant_to_patient_distribution_in_today_task_for_operator_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.operator_login==operator_login)
         ].head(task_amount),
        
       
        ]).reset_index(drop=True)

initial_task_for_today.shape[0]

CPU times: total: 2.58 s
Wall time: 2.57 s


17547

In [121]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=True).head(10)

,,emiasid
mo,operator_login,
ГП 107,pomoshnik_pdn131,1
ГП 9,pomoshnik_pdn25,1
ГП 52,pomoshnik_pdn103,2
ГП 109,pomoshnik_pdn155,5
ГП 23,pomoshnik_pdn104,5
ГП 66,pomoshnik_pdn54,7
ГП 45,pomoshnik_pdn82,9
КДЦ 6,pomoshnik_pdn60,15
ГП 9,pomoshnik_pdn243,29


In [122]:
initial_task_for_today \
.groupby(['mo', 'operator_login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'])\
.sort_values(by='emiasid', ascending=False).head(10)

emiasid
mo    operator_login           
ГП 45 pomoshnik_pdn293       90
ГП 64 pomoshnik_pdn79        90
ГП 6  pomoshnik_pdn182       90
ГП 62 pomoshnik_pdn107       90
      pomoshnik_pdn108       90
      pomoshnik_pdn267       90
      Помощник 4             90
ГП 64 pomoshnik_pdn105       90
      pomoshnik_pdn17        90
      pomoshnik_pdn45        90

In [123]:
rename_mo = \
{'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62', 
'ГБУЗ "ГП № 52 ДЗМ"': 'ГП 52', 'ГБУЗ "ГП 67 ДЗМ"': 'ГП 67', 'ГБУЗ ГП 68 ДЗМ': 'ГП 68', 'ГБУЗ ГП 175 ДЗМ': 'ГП 175',
'ГБУЗ ГП № 209 ДЗМ': 'ГП 209',  'ГБУЗ ГП 212 ДЗМ': 'ГП 212', 'ГБУЗ ГП 19 ДЗМ': 'ГП 19', 
'ГБУЗ ГП №166 ДЗМ': 'ГП 166', 'ГБУЗ ГП 9 ДЗМ': 'ГП 9', 'ГБУЗ ГП 69 ДЗМ': 'ГП 69',
'ГБУЗ "ГП № 3 ДЗМ"': 'ГП 3', 'ГБУЗ ГП 5 ДЗМ': 'ГП 5', 'ГБУЗ ГП 46 ДЗМ': 'ГП 46', 'ГБУЗ ГП №8 ДЗМ': 'ГП 8', 
'ГБУЗ "ГП № 45 ДЗМ"': 'ГП 45'}

In [124]:
mo_list = set(rename_mo.values())

In [125]:
initial_task_for_today_present_assistants = \
initial_task_for_today[~initial_task_for_today.operator_login.isin(absent_assistants.operator_login)]

In [126]:
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.operator_login)].shape[0]

4575

In [127]:
initial_task_for_today_present_assistants.shape[0]

12972

In [128]:
initial_task_for_today_present_assistants.groupby('operator_login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
operator_login,
pomoshnik_pdn101,90
pomoshnik_pdn29,90
pomoshnik_pdn45,90
pomoshnik_pdn43,90
pomoshnik_pdn41,90
...,...
pomoshnik_pdn155,5
pomoshnik_pdn104,5
pomoshnik_pdn103,2


In [129]:
initial_task_for_today_absent_assistants =  \
initial_task_for_today[initial_task_for_today.operator_login.isin(absent_assistants.operator_login)] \
[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]

In [130]:
initial_task_for_today_absent_assistants.columns

Index(['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month'], dtype='object')

In [131]:
new_doctor_patients = actual_patients[actual_patients.problem == 'Новый врач']

In [132]:
new_doctor_patients['priority'] = 6

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7380\1491646637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_doctor_patients['priority'] = 6


In [133]:
new_doctor_patients = new_doctor_patients[['emiasid', 'status', 'old_comment', 'mo', 'priority', 'month']]
new_doctor_patients.shape[0]

9357

In [134]:
additional_task_for_present_assistants = \
pd.concat([
    new_doctor_patients,
    initial_task_for_today_absent_assistants
]).reset_index(drop=True)
additional_task_for_present_assistants.shape[0]

13932

In [135]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [136]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']) \
.agg({'emiasid': 'count', 'operator_login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [137]:
distribution_count['total_delta'] = distribution_count['operator_login'] * task_amount - distribution_count['emiasid']

In [138]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [139]:
additional_task_for_present_assistants

,emiasid,status,old_comment,mo,priority,month
0,21346047,Ни разу не звонили,"Не в Москве, сентябрь",ГП 19,6,august
1,1943222591,Ни разу не звонили,"не в Москве, сентябрь",ГП 11,6,august
2,10809670,Не дозвонились,"Не в москве, сентябрь",ГП 170,6,august
3,19125318,Ни разу не звонили,"не в москве , сентябрь",ГП 23,6,august
4,16536830,Разговор состоялся,NaN,ГП 170,6,august
...,...,...,...,...,...,...
13927,10784961,Напомнить,NaN,ГП 23,4,september
13928,10945571,Напомнить,NaN,ГП 23,4,september
13929,16154509,Напомнить,NaN,ГП 23,4,september
13930,21184970,Напомнить,NaN,ГП 23,4,september


In [140]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [141]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_before

13932

In [142]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('operator_login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < task_amount and \
    final_task_predistribution[(final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.operator_login.isna()) & (final_task_predistribution.mo == mo)
        ].index[0]
        
        final_task_predistribution = \
        final_task_predistribution.sort_values(by=['priority', 'calls_amount', 'last_call_date'])

        final_task_predistribution.loc[empty_login_in_mo_index, 'operator_login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 45.5 s
Wall time: 45.7 s


In [143]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.operator_login.isna()].shape[0]
empty_login_patients_amount_after

12332

In [144]:
check_table = distribution_count.merge(additional_task_for_present_assistants_by_mo,left_index=True, right_index=True) \
[['total_delta', 'emiasid_y']]

In [145]:
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'emiasid_y']

check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'total_delta']

In [146]:
real_change_of_emty_login_patients_amount = check_table.real_addition.sum()
real_change_of_emty_login_patients_amount

1600.0

In [147]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [148]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.operator_login.isna()] \
.copy().reset_index(drop=True)

In [149]:
final_task_distribution.shape[0]

14572

In [150]:
final_task_distribution.status.value_counts()

Не дозвонились                    8575
Перенос звонка на другой день     2747
Напомнить                         1582
Запишется самостоятельно          1150
Ни разу не звонили                 309
Будет в Москве в другом месяце      58
Разговор состоялся                  41
Отсутствует номер                   32
Не нравится врач                    19
Перезвонить сегодня                 19
Другой врач в МО                    13
Не нравится МО                       6
Госпитализирован                     5
Пациент отрицает диагноз             5
Иная МО                              4
Отказ                                4
Патронаж                             2
Онкология                            1
Name: status, dtype: int64

In [151]:
final_task_distribution.shape[0]

14572

In [ ]:
final_task_distribution.columns

In [ ]:
final_task_distribution.groupby('operator_login').agg({'emiasid': 'count'}).sort_values(by=['emiasid'], ascending=False).head(30)

In [ ]:
final_task_distribution['emiasid'] = final_task_distribution['emiasid']

In [ ]:
final_task_distribution = final_task_distribution.drop_duplicates(subset=['emiasid'], keep='first')
final_task_distribution.shape[0]

In [ ]:
final_task_distribution.status.value_counts()

In [ ]:
final_task_distribution.last_call_date.value_counts()

In [ ]:
final_task_distribution = final_task_distribution.drop_duplicates(subset=['emiasid'], keep='first')
final_task_distribution.shape[0]

In [ ]:
final_task_distribution.operator_login = final_task_distribution.operator_login.apply(lambda x: x + '_n2')

In [152]:
final_task_distribution.operator_login.value_counts()

pomoshnik_pdn62     90
pomoshnik_pdn247    90
pomoshnik_pdn20     90
pomoshnik_pdn147    90
pomoshnik_pdn38     90
                    ..
Помощник 4          90
pomoshnik_pdn180    90
pomoshnik_pdn91     90
pomoshnik_pdn199    86
pomoshnik_pdn265    86
Name: operator_login, Length: 162, dtype: int64

In [153]:
final_task_distribution[final_task_distribution.month == 'august']\
.to_excel(rf'august_task_for_today_{today_y_m_d}.xlsx', index=False)

In [154]:
final_task_distribution[final_task_distribution.month == 'september']\
.to_excel(rf'september_task_for_today_{today_y_m_d}.xlsx', index=False)

In [155]:
final_task_distribution.to_excel(rf'full_task_for_today_{today_y_m_d}.xlsx', index=False)

In [ ]:
tomorrow_record_patients[tomorrow_record_patients.emiasid.isin(final_task_distribution.emiasid)]

In [ ]:
final_task_distribution.